# 영화 추천 시스템

 - 무비렌즈 태그 데이터를 중심으로 구현

### 라이브러리

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## 1. 영화 정보 핸들링

In [2]:
movie = pd.read_csv('movies.csv')
score = pd.read_csv('genome-scores.csv')   # 영화와 태그의 연관성 점수
tags = pd.read_csv('tags.csv')
tags_genome = pd.read_csv('genome-tags.csv')

df1 = pd.merge(movie, tags, on='movieId', how='inner')
df1.groupby('movieId').count()

,title,genres,userId,tag,timestamp
movieId,,,,,
1,697,697,697,697,697
2,180,180,180,180,180
3,29,29,29,29,29
4,11,11,11,11,11
5,24,24,24,24,24
...,...,...,...,...,...
208813,1,1,1,1,1
208933,2,2,2,2,2
209035,6,6,6,6,6


In [3]:
movie_tag = df1[['movieId', 'title', 'tag', 'genres']]
genre_tag = df1[['genres', 'tag']]
genre_tag

genre_tag

genres_list = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary',
'Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller', 'War', 'Western']


for i in genres_list:
    globals()['tag_' + i] = genre_tag[genre_tag['genres'].str.contains(i)]
    globals()['tag_'+ i + '_id'] = globals()['tag_' + i]['tag'].unique().tolist()

type(tag_Adventure_id)

list

In [4]:
genre_tag

,genres,tag
0,Adventure|Animation|Children|Comedy|Fantasy,Owned
1,Adventure|Animation|Children|Comedy|Fantasy,imdb top 250
2,Adventure|Animation|Children|Comedy|Fantasy,Pixar
3,Adventure|Animation|Children|Comedy|Fantasy,Pixar
4,Adventure|Animation|Children|Comedy|Fantasy,time travel
...,...,...
1093355,(no genres listed),Philadelphia
1093356,(no genres listed),private school
1093357,(no genres listed),quaker
1093358,(no genres listed),racism


In [5]:
tag_Action_id

['imdb top 250',
 'great acting',
 'realistic action',
 'suspense',
 'Al Pacino',
 'atmospheric',
 'bank robbery',
 'crime',
 'Robert De Niro',
 'tense',
 'Val Kilmer',
 'bank job',
 'dialogue',
 'heist',
 'heist movie',
 'long',
 'Los Angeles',
 'Michael Mann',
 'visceral',
 'bleak',
 'slick',
 'somber',
 'witty',
 'betrayal',
 'cat and mouse',
 'chase',
 'crime epic',
 'criminal mastermind',
 'cult film',
 'detective',
 'ex-con',
 'gang',
 'honor',
 'loner',
 'los angeles',
 'murder',
 'neo-noir',
 'obsession',
 'one last job',
 'robbery',
 'thief',
 'Recommendz Top Pick',
 'gunfight',
 '1',
 'overrated',
 'philosophy',
 'Natalie Portman',
 'soundtrack',
 'al pacino',
 'electronic soundtrack',
 'robert de niro',
 'slow paced',
 'blu-ray',
 'mine',
 'realistic',
 'cliched',
 'dumbed down',
 'hollywoodization',
 'lame',
 '7.5-FilmAffinity',
 'es un poc de lo de sempre',
 'too long',
 'Ashley Judd',
 'Bechdel Test:Fail',
 'ensemble cast',
 'police',
 'relationships',
 'individualism',
 

In [6]:
import zipfile
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# 각 장르를 하나의 문서로 취급하여 벡터표현으로 학습
# 학습 데이터는 각 장르의 모든 태그 데이터.
# 태그 데이터의 말뭉치를 일종의 description의 tokenizing 된 단어들로 취급.


## 2. 유저 정보 핸들링

In [7]:
rating = pd.read_csv('ratings.csv')

total_df = pd.merge(movie, rating, on='movieId', how='inner')
# 유저는 162541명이 존재
len_userId = rating['userId'].unique()

# for i in range(162541):
#     globals()['userId_'+ i] = rating[rating['userId'].str.contains(str(i))].values.tolist()

total_df.groupby('userId').count()

,movieId,title,genres,rating,timestamp
userId,,,,,
1,70,70,70,70,70
2,184,184,184,184,184
3,656,656,656,656,656
4,242,242,242,242,242
5,101,101,101,101,101
...,...,...,...,...,...
162537,101,101,101,101,101
162538,154,154,154,154,154
162539,47,47,47,47,47


In [8]:
for j in range(1, 101, 1):
    globals()['userId_'+ str(j)] = total_df.loc[(total_df['userId'] == j)]

In [9]:
userId_2

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,3.5,1141415820
674928,62,Mr. Holland's Opus (1995),Drama,2,0.5,1141417130
842066,110,Braveheart (1995),Action|Drama|War,2,5.0,1141416589
1015083,150,Apollo 13 (1995),Adventure|Drama|IMAX,2,4.0,1141415790
1063460,151,Rob Roy (1995),Action|Drama|Romance|War,2,4.5,1141415643
...,...,...,...,...,...,...
19576205,33794,Batman Begins (2005),Action|Crime|IMAX,2,5.0,1141416640
19640187,34162,Wedding Crashers (2005),Comedy|Romance,2,0.5,1141416317
19694838,35836,"40-Year-Old Virgin, The (2005)",Comedy|Romance,2,0.5,1141417449
19727001,36527,Proof (2005),Drama,2,3.5,1141417382


In [10]:
tags

for k in range(1, 1001, 1):
    globals()['userId_tags_' + str(k)] = tags.loc[(tags['userId'] == k)]

In [11]:
userId_tags_4

,userId,movieId,tag,timestamp
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455
5,4,44665,unreliable narrators,1573943619
6,4,115569,tense,1573943077
7,4,115713,artificial intelligence,1573942979
8,4,115713,philosophical,1573943033
9,4,115713,tense,1573943042
10,4,148426,so bad it's good,1573942965
11,4,164909,cliche,1573943721


In [12]:
# 유저가 본 영화들의 정보를 userId_n 으로 동적변수로 담아서 각각의 DataFrame으로 생성.
# 생성한 df에서 userId에는 해당 유저가 본 영화와 평점에 대한 모든 정보가 담김.
# userId_tags_n 에서는 해당 유저가 남긴 태그 정보를 모두 모았는데 태그 정보를 남기지 않은 유저들도 많기 때문에 중간에 빈 df들도 많다.

# 2. 영화 추천 시스템 아키텍쳐

 - 각 장르에 모든 태그 정보를 담은 데이터프레임을 input
 - doc2vec을 통해서 각 장르가 머신러닝으로 사용할 수 있는 형태(Document Representation)로 정제.
 - vector space에서 표현된 18개 장르의 vector를 맵핑
 - 유저 정보에서 해당 유저가 관람한 모든 영화 정보를 추출.
 - 유저 한 명이 자주 관람하는 영화 장르를 기준으로 유저를 vector space 맵핑.
 - genres vector 와 user vector 사이의 유사도 계산으로 새롭게 들어온 유저에게 추천하는 리스트를 뽑기 위한 조건 추출.
 - 장르 계산으로 해당 유저가 관심 있는 장르를 판별하여 가장 가까운 장르 3가지를 추출.
 - 유저 계산으로 해당 유저 시청기록에 근거하여 맵핑된 위치에서 근방에 위치한 다른 유저의 영화 시청기록에 근거하여 추천.
 - 추천 조건 = 근처 유저의 4.0 이상의 평점 & 관심 장르 3가지 or 해당 장르에서 최근 년도에 개봉한 4.0 이상의 영화

<img src="그림1_문서 분류 과정.png" width=700, height=500>

 <span style="font-size:130%"> 1. 태그 정보를 수집하여 핸들링 과정을 거쳐 특징선택 부분까지 진행. 태그 데이터이기 때문에 토큰, 불용어, 어간 관련 작업 불필요</span>

  <span style="font-size:130%"> 2. 문서 벡터 표현을 위해서 Doc2vec 모델을 사용하여 각 장르명을 문서명으로 설정하여 태그 데이터를 학습.</span>

In [13]:
new_df = rating.groupby('movieId', as_index=False).mean()
total_tag = pd.merge(new_df[['movieId', 'rating']], tags, on='movieId', how='inner')
for i in range(1, 101, 1):
    doc_tag = total_tag.loc[(total_tag['movieId'] == i)].values.tolist()
total_tag

,movieId,rating,userId,tag,timestamp
0,1,3.893708,791,Owned,1515175493
1,1,3.893708,1048,imdb top 250,1172144394
2,1,3.893708,1361,Pixar,1216146311
3,1,3.893708,3164,Pixar,1223304727
4,1,3.893708,3164,time travel,1223304729
...,...,...,...,...,...
1083523,209063,4.000000,96399,Philadelphia,1574021180
1083524,209063,4.000000,96399,private school,1574021158
1083525,209063,4.000000,96399,quaker,1574021197
1083526,209063,4.000000,96399,racism,1574021194


In [14]:
tagged_data = [TaggedDocument(words=globals()['tag_' + genre + '_id'], tags=[genre]) for genre in genres_list]
tagged_data[0]

TaggedDocument(words=['imdb top 250', 'great acting', 'realistic action', 'suspense', 'Al Pacino', 'atmospheric', 'bank robbery', 'crime', 'Robert De Niro', 'tense', 'Val Kilmer', 'bank job', 'dialogue', 'heist', 'heist movie', 'long', 'Los Angeles', 'Michael Mann', 'visceral', 'bleak', 'slick', 'somber', 'witty', 'betrayal', 'cat and mouse', 'chase', 'crime epic', 'criminal mastermind', 'cult film', 'detective', 'ex-con', 'gang', 'honor', 'loner', 'los angeles', 'murder', 'neo-noir', 'obsession', 'one last job', 'robbery', 'thief', 'Recommendz Top Pick', 'gunfight', '1', 'overrated', 'philosophy', 'Natalie Portman', 'soundtrack', 'al pacino', 'electronic soundtrack', 'robert de niro', 'slow paced', 'blu-ray', 'mine', 'realistic', 'cliched', 'dumbed down', 'hollywoodization', 'lame', '7.5-FilmAffinity', 'es un poc de lo de sempre', 'too long', 'Ashley Judd', 'Bechdel Test:Fail', 'ensemble cast', 'police', 'relationships', 'individualism', 'heists', 'complex characters', 'setting:LA', '

In [15]:
max_epochs = 10

model = Doc2Vec(
    window=10,
    vector_size=150,
    alpha=0.025,
    min_alpha=0.025,
    min_count=2,
    dm=1,
    negative=5,
    seed=9999
)

model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs
    )

    model.alpha -= 0.002
    model.min_alpha = model.alpha

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [16]:
recommend_tag = model.wv.most_similar_cosmul(userId_tags_4['tag'].values.tolist())

In [17]:
recommend_list = [recommend_tag[i][0] for i in range(10)]
recommend_list

['atmospheric',
 'Nudity (Topless)',
 'criminal mastermind',
 'cult film',
 'dark humor',
 'Las Vegas',
 'disturbing',
 'amazing cast',
 'tanks',
 'surreal']

In [20]:
result_recommend = movie_tag.loc[movie_tag['tag'].isin(recommend_list)]
pre_result = total_tag.loc[total_tag['movieId'].isin(result_recommend['movieId'].unique().tolist())]
result = pre_result.groupby('movieId', as_index=False).mean()
result_movieId = result.loc[result['rating'] >= 4.5]
movie.loc[movie['movieId'].isin(result_movieId['movieId'].values.tolist())]

,movieId,title,genres
36864,151367,Twisted Seduction (2010),Comedy|Romance|Thriller
44729,168850,Enemy Empire (2013),Adventure|Fantasy|Sci-Fi|Western
50049,180163,Stanley Pickle (2010),(no genres listed)
50436,181027,The House in Nightmare Park (1973),Comedy|Horror
56565,194434,Adrenaline (1990),(no genres listed)
61926,207287,ami. exe (2017),Sci-Fi
62256,208465,Jallikattu (2019),Action|Crime|Drama|Thriller
